In [1]:
import sentencepiece as spm
import pandas as pd
import numpy as np
import time

from sklearn.feature_extraction.text import *
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

import pickle

import warnings
warnings.filterwarnings('ignore')

sp = spm.SentencePieceProcessor()
sp.Load('../mongolian_bert_sentencepiece/mn_uncased.model')

def sp_tokenize(w):
    return sp.EncodeAsPieces(w)

" ".join(sp_tokenize('Мөнгөө тушаачихсаныхаа дараа мэдэгдээрэй'.lower()))

'▁мөнгөө ▁тушаа чихсан ыхаа ▁дараа ▁мэдэгд ээрэй'

In [2]:
df_path = '../../main_dataset/'
trained_model_and_fitted_encoder_path = '../models/'
plots_path = '../plots'

with open('../dataset/word_index.pickle', 'rb') as handle:
    word_index = pickle.load(handle)
    
with open('../dataset/reversed_word_index.pickle', 'rb') as handle:
    reversed_word_index = pickle.load(handle)


url = 'https://drive.google.com/file/d/1hm5SD4GMEael6taLagdBc33Wz9ccoflq/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
with open(path, 'rb') as handle:
    df = pickle.load(handle)

In [3]:
MAX_LEN = 512

import itertools

for item in df:
    item[0] = list(itertools.chain(*item[0]))[:MAX_LEN]

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.1, random_state = 999)

train_data_words  = [i[0] for i in train]
train_label_words = [i[1] for i in train]
test_data_words   = [i[0] for i in test ]
test_label_words  = [i[1] for i in test ]

In [5]:
train_words = []
test_words = []

for item in train_data_words:
    sentence = " ".join(item)
    train_words.append(sentence)
    
for item in test_data_words:
    sentence = " ".join(item)
    test_words.append(sentence)

In [6]:
labels = list(set(test_label_words))
labels

['Өргөдөл', 'Гомдол', 'Шүүмжлэл', 'Талархал', 'Санал хүсэлт']

In [7]:
text_clf = Pipeline([('vect', CountVectorizer(tokenizer=sp_tokenize, lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, max_iter=5, random_state=0))])

t = time.time()
text_clf = text_clf.fit(train_words, train_label_words)
t = time.time()-t
print("Training time in seconds: ", t)

t = time.time()
predicted = text_clf.predict(test_words)
t = time.time()-t
print("Prediction time in seconds: ", t)

print("Feature count:", len(text_clf.named_steps['vect'].vocabulary_))
print("Classifier accuracy: ", np.mean(predicted == test_label_words))

Training time in seconds:  7.195267200469971
Prediction time in seconds:  0.6535961627960205
Feature count: 25746
Classifier accuracy:  0.7591212240094155


In [8]:
# Sample texts to classify intent
to_pred = ['Эмийн найдвартай байдлыг хангаж өгнө үү . Эм чимээгүй дайн үүсгэж байна.Ямарч баталгаа алга жнь: хугацаа нь дууссан гэх мэт',
          'Сайн байна уу? Та бүхэндээ баярлалаа. Миний санал: өмнөх жилийн удаан буудайн борлуулалтын үнэ 350000 төгрөг байсан. Энэ жил яагаад бууруулахгүй гэчихээд бага үнээр авав?',
          'Засгийн газраас хэрэглүүлж байгаа 100000 орон сууцны хөрөнгөө төрийн банкинд өгөөч ээ байрандаа ормоор байна',
          'Энэ засгийн газар хүн бүрд сая төгрөг өгөх болох уу?']

text_clf.predict([to_pred[0]])

array(['Санал хүсэлт'], dtype='<U12')

In [13]:
to_pred_testing = [
    'Би шинэ апп татаад орох гэхээр орохгүй шууд гарчихаад',
    'Танай сүлжээ болохгүй байх шиг байна хэзээ хэвийн болох вэ',
    'одоо тэр мөнгөө яаж авч байж хаах уу',
    'Е-код интернэтээр худалдан авалтын код онлайнаар авч болох уу'
]

for entry in to_pred_testing:
    print('Text:', entry)
    print('Pred:', text_clf.predict([entry])[0], '\n')

Text: Би шинэ апп татаад орох гэхээр орохгүй шууд гарчихаад
Pred: Санал хүсэлт 

Text: Танай сүлжээ болохгүй байх шиг байна хэзээ хэвийн болох вэ?
Pred: Санал хүсэлт 

Text: одоо тэр мөнгөө яаж авч байж хаах уу
Pred: Санал хүсэлт 

Text: Е-код интернэтээр худалдан авалтын код онлайнаар авч болох уу
Pred: Санал хүсэлт 

